In [18]:
import json
import pickle
import random
from collections import Counter
import jieba

In [2]:
data_handle = open('./train.zh', 'r')
data_rows=[]
for row in data_handle:
    data_rows.append(row)

In [4]:
print(len(data_rows), random.choice(data_rows), row, type(row), row[4])
with open('all_data.json', 'w') as f:
    json.dump(data_rows, f)

9903244 我丧失了原有的信心。 
 我们约了8:30。不是，是9:00。
 <class 'str'> 8


In [5]:
with open('all_data.json', 'r') as f:
    all_data = json.load(f)

In [21]:
rand_sample = random.choice(all_data)
print(len(all_data), rand_sample, list(rand_sample), jieba.lcut(rand_sample))

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.242 seconds.
Prefix dict has been built succesfully.


9903244 我觉得嘛你对结局有种病态的纠结。
 ['我', '觉', '得', '嘛', '你', '对', '结', '局', '有', '种', '病', '态', '的', '纠', '结', '。', '\n'] ['我', '觉得', '嘛', '你', '对', '结局', '有种', '病态', '的', '纠结', '。', '\n']


In [16]:
class Vocab:
    def __init__(self, stop_words_set=set()):
        self.word_counter=Counter()
        self.stopwords=stop_words_set
        
    def insert_sentence(self, sentence):#remove stop words?
        self.word_counter.update(sentence)
    
    def vocab_analyse(self, count_clip=5):
        removed_word_cnt=0
        removed_token_cnt=0

        all_word_cnt=0
        all_token_cnt=0
        
        for x in self.word_counter:
            all_word_cnt+=self.word_counter[x]
            all_token_cnt+=1
            
            if self.word_counter[x]<count_clip:
                removed_word_cnt+=self.word_counter[x]
                removed_token_cnt+=1
            
        print('in corpus, %2.4f%%(%s/%s) words will be removed if count_clip is %s.'%(100*removed_word_cnt/all_word_cnt, 
                                                                                      removed_word_cnt, all_word_cnt, count_clip))
        print('in vocabulary, %2.4f%%(%s/%s) tokens will be removed if count_clip is %s.'%(100*removed_token_cnt/all_token_cnt, 
                                                                                           removed_token_cnt, all_token_cnt, count_clip))
    
    def build_vocab(self, count_clip=0):
        self.token2word=['sos', 'padding', 'eos', 'low_freq', 'mask']
        self.word2token={'sos':0, 'padding':1, 'eos':2, 'low_freq':3, 'mask':4}
        self.token2count={'sos':99, 'padding':99, 'eos':99, 'low_freq':99, 'mask':99}
        
        for x in self.word_counter:
            if self.word_counter[x] >= count_clip:
                self.token2word.append(x)
                self.word2token[x]=len(self.token2word)-1
                self.token2count[x] = self.word_counter[x]
                
    def vocab_save(self, path):
        with open(path, 'wb') as f:
            pickle.dump(self, f)

In [24]:
all_data_lcut = [1]*len(all_data)
data_size = len(all_data_lcut)

for idx, data in enumerate(all_data):
    all_data_lcut[idx]=jieba.lcut(data)
    
    if idx%int(data_size/10)==1:
        print(idx)

1
990325
1980649
2970973
3961297
4951621
5941945
6932269
7922593
8912917
9903241


In [26]:
vocab = Vocab()
data_size = len(all_data_lcut)
for idx, data in enumerate(all_data_lcut):
    vocab.insert_sentence(data)
    
    if idx%int(data_size/10)==1:
        print(idx)

1
990325
1980649
2970973
3961297
4951621
5941945
6932269
7922593
8912917
9903241


In [29]:
vocab.vocab_analyse(count_clip=20)
vocab.build_vocab(count_clip=20)
print(len(vocab.word2token))
vocab.vocab_save('vocab.pk')

in corpus, 1.4255%(1766026/123885822) words will be removed if count_clip is 20.
in vocabulary, 85.3907%(576499/675131) tokens will be removed if count_clip is 20.
98637


In [38]:
class Tokenizer:
    def __init__(self, vocab, stop_words = set()):
        self.stop_words = stop_words
        self.vocab = vocab
    
    
    def sentences_length(self, sentences):
        self.sentences_length=[]
        for sent in sentences:
            #sent = list(sentence)
            word_cnt=0
            for word in sent:
                if word not in self.stop_words:
                    word_cnt+=1
            self.sentences_length.append(word_cnt)
            
            
    def sentences_length_analyse(self, sentence_length_clip=30):
        sent_cnt=0
        for sent_length in self.sentences_length:
            if sent_length <= sentence_length_clip:
                sent_cnt+=1
        print('%2.4f%%(%s/%s) sentences\'length are shorter than %s.'%(100*sent_cnt/len(self.sentences_length), 
                                                                     sent_cnt, len(self.sentences_length), sentence_length_clip))
        
    def sentence_tokenize(self, sentence, max_length):
        tokenized_sent=[]
        for word in sentence:
            if word not in self.stop_words and len(tokenized_sent)<max_length:
                if word in self.vocab.word2token:
                    tokenized_sent.append(self.vocab.word2token[word])
                else:
                    tokenized_sent.append(self.vocab.word2token['low_freq'])
        real_len = len(tokenized_sent)
        
        while(len(tokenized_sent)<max_length):
            tokenized_sent.append(self.vocab.word2token['padding'])
            
        return tokenized_sent, real_len

In [39]:
with open('vocab.pk', 'rb') as f:
    vocab = pickle.load(f)

stop_words = {'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'z', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
              'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
              '1', '2', '3', '4', '5', '6', '7', '8', '9', '.', '*', '&', '^', '#', '$', '@', '￥', '~', '+', '-', '=', '\n', ' '}
tokenizer = Tokenizer(vocab, stop_words= stop_words)
tokenizer.sentences_length(all_data_lcut)

In [40]:
tokenizer.sentences_length_analyse(sentence_length_clip=25)

94.7130%(9379655/9903244) sentences'length are shorter than 25.


In [41]:
tokenized_sents=[]
tokenized_sents_real_len=[]
data_size = len(all_data_lcut)

for idx, sent in enumerate(all_data_lcut):
    tokenized_sent, real_len = tokenizer.sentence_tokenize(sent, max_length=25)
    tokenized_sents.append(tokenized_sent)
    tokenized_sents_real_len.append(real_len)
    
    if idx%int(data_size/10)==1:
        print(idx)

1
990325
1980649
2970973
3961297
4951621
5941945
6932269
7922593
8912917
9903241


In [72]:
idx = random.randint(0,len(tokenized_sents))
print(len(tokenized_sents), idx, tokenized_sents[idx], len(tokenized_sents[idx]), tokenized_sents_real_len[idx])

9903244 7091984 [14058, 59835, 86185, 80168, 97088, 32932, 83094, 88055, 16218, 2352, 43062, 80168, 42385, 7489, 11659, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 25 15


In [181]:
with open('tokenized_sentences.pk', 'wb') as f:
    pickle.dump(tokenized_sents, f)
with open('tokenized_sentences_real_length.pk', 'wb') as f:
    pickle.dump(tokenized_sents_real_len, f)